# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 9:04AM,238834,19,AVET06072022,"Avet Pharmaceuticals Labs, Inc",Released
1,Jun 6 2022 9:03AM,238832,10,0085740495,ISDIN Corporation,Released
2,Jun 6 2022 9:03AM,238832,10,0085740497,ISDIN Corporation,Released
3,Jun 6 2022 9:03AM,238832,10,0085740287,ISDIN Corporation,Released
4,Jun 6 2022 9:03AM,238832,10,0085740289,ISDIN Corporation,Released
5,Jun 6 2022 9:03AM,238832,10,0085740291,ISDIN Corporation,Released
6,Jun 6 2022 9:03AM,238832,10,0085740259,ISDIN Corporation,Released
7,Jun 6 2022 9:03AM,238832,10,0085740290,ISDIN Corporation,Released
8,Jun 6 2022 9:03AM,238832,10,0085740295,ISDIN Corporation,Released
9,Jun 6 2022 9:03AM,238832,10,0085740297,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,238829,Released,27
15,238830,Released,12
16,238832,Released,39
17,238833,Released,2
18,238834,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238829,NaN,27.0
238830,NaN,12.0
238832,NaN,39.0
238833,NaN,2.0
238834,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238818,0.0,1.0
238819,6.0,23.0
238820,5.0,9.0
238821,0.0,2.0
238822,1.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238818,0,1
238819,6,23
238820,5,9
238821,0,2
238822,1,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238818,0,1
1,238819,6,23
2,238820,5,9
3,238821,0,2
4,238822,1,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238818,,1
1,238819,6,23
2,238820,5,9
3,238821,,2
4,238822,1,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc"
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation
40,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation
67,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC"
69,Jun 6 2022 8:57AM,238830,10,Beach Products Inc
81,Jun 6 2022 8:56AM,238826,10,ISDIN Corporation
143,Jun 6 2022 8:55AM,238828,12,Hush Hush
190,Jun 6 2022 8:51AM,238827,19,"GUSA Granules USA, Inc."
191,Jun 6 2022 8:46AM,238825,10,Emerginnova
193,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc",,1
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation,,39
2,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation,,27
3,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC",,2
4,Jun 6 2022 8:57AM,238830,10,Beach Products Inc,,12
5,Jun 6 2022 8:56AM,238826,10,ISDIN Corporation,,62
6,Jun 6 2022 8:55AM,238828,12,Hush Hush,,47
7,Jun 6 2022 8:51AM,238827,19,"GUSA Granules USA, Inc.",,1
8,Jun 6 2022 8:46AM,238825,10,Emerginnova,,2
9,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,,58


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc",1,
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation,39,
2,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation,27,
3,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC",2,
4,Jun 6 2022 8:57AM,238830,10,Beach Products Inc,12,
5,Jun 6 2022 8:56AM,238826,10,ISDIN Corporation,62,
6,Jun 6 2022 8:55AM,238828,12,Hush Hush,47,
7,Jun 6 2022 8:51AM,238827,19,"GUSA Granules USA, Inc.",1,
8,Jun 6 2022 8:46AM,238825,10,Emerginnova,2,
9,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,58,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc",1,
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation,39,
2,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation,27,
3,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC",2,
4,Jun 6 2022 8:57AM,238830,10,Beach Products Inc,12,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc",1,NaN
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation,39,NaN
2,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation,27,NaN
3,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC",2,NaN
4,Jun 6 2022 8:57AM,238830,10,Beach Products Inc,12,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 9:04AM,238834,19,"Avet Pharmaceuticals Labs, Inc",1,0.0
1,Jun 6 2022 9:03AM,238832,10,ISDIN Corporation,39,0.0
2,Jun 6 2022 9:03AM,238829,10,ISDIN Corporation,27,0.0
3,Jun 6 2022 9:02AM,238833,10,"Snap Medical Industries, LLC",2,0.0
4,Jun 6 2022 8:57AM,238830,10,Beach Products Inc,12,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2149444,244,1.0
12,238828,47,0.0
19,1194121,36,11.0
22,238818,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2149444,244,1.0
1,12,238828,47,0.0
2,19,1194121,36,11.0
3,22,238818,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,244,1.0
1,12,47,0.0
2,19,36,11.0
3,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,244.0
1,12,Released,47.0
2,19,Released,36.0
3,22,Released,1.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,22
Status,,,,
Executing,1.0,0.0,11.0,0.0
Released,244.0,47.0,36.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,22
0,Executing,1.0,0.0,11.0,0.0
1,Released,244.0,47.0,36.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,22
0,Executing,1.0,0.0,11.0,0.0
1,Released,244.0,47.0,36.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()